# Reinforce Algorthim Training

This notebook implements the REINFORCE (Monte Carlo Policy Gradient) algorithm to train an agent for the fire-rescue mission environment.

In [ ]:
# %% Cell 1: Imports and Setup
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import matplotlib.pyplot as plt
from collections import deque
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from environment.custom_env import FireRescueEnv

# Set random seeds
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create directories
os.makedirs("../models/reinforce", exist_ok=True)
os.makedirs("../models/reinforce/best_model", exist_ok=True)

In [ ]:
# %% Cell 2: Policy Network Architecture
class PolicyNetwork(nn.Module):
    """Neural network for policy approximation in REINFORCE."""
    
    def __init__(self, state_dim, action_dim, hidden_dims=[128, 64]):
        super(PolicyNetwork, self).__init__()
        
        layers = []
        prev_dim = state_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.1)
            ])
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, action_dim))
        layers.append(nn.Softmax(dim=-1))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, state):
        return self.network(state)

In [ ]:
# %% Cell 3: REINFORCE Agent Implementation
class REINFORCEAgent:
    """REINFORCE algorithm implementation."""
    
    def __init__(self, state_dim, action_dim, learning_rate=0.001, 
                 gamma=0.99, hidden_dims=[128, 64], entropy_coef=0.01):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.entropy_coef = entropy_coef
        
        self.policy = PolicyNetwork(state_dim, action_dim, hidden_dims).to(device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=learning_rate)
        
        self.saved_log_probs = []
        self.rewards = []
        self.entropies = []
    
    def select_action(self, state, training=True):
        """Select action using current policy."""
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        probs = self.policy(state)
        
        if training:
            m = Categorical(probs)
            action = m.sample()
            self.saved_log_probs.append(m.log_prob(action))
            self.entropies.append(m.entropy())
            return action.item()
        else:
            return torch.argmax(probs).item()
    
    def store_reward(self, reward):
        """Store reward for current step."""
        self.rewards.append(reward)
    
    def update(self):
        """Update policy using REINFORCE algorithm."""
        if len(self.rewards) == 0:
            return 0.0
        
        # Calculate discounted returns
        returns = []
        G = 0
        for r in reversed(self.rewards):
            G = r + self.gamma * G
            returns.insert(0, G)
        
        # Normalize returns
        returns = torch.tensor(returns, dtype=torch.float32).to(device)
        if len(returns) > 1:
            returns = (returns - returns.mean()) / (returns.std() + 1e-8)
        
        # Calculate policy loss
        policy_loss = []
        for log_prob, G, entropy in zip(self.saved_log_probs, returns, self.entropies):
            policy_loss.append(-log_prob * G - self.entropy_coef * entropy)
        
        # Optimize
        self.optimizer.zero_grad()
        loss = torch.stack(policy_loss).sum()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.policy.parameters(), 1.0)
        self.optimizer.step()
        
        # Clear episode data
        loss_value = loss.item()
        self.saved_log_probs = []
        self.rewards = []
        self.entropies = []
        
        return loss_value
    
    def save(self, path):
        """Save model."""
        torch.save({
            'policy_state_dict': self.policy.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
        }, path)
    
    def load(self, path):
        """Load model."""
        checkpoint = torch.load(path, map_location=device)
        self.policy.load_state_dict(checkpoint['policy_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
# %% Cell 4: Training Function
def train_reinforce(env, agent, num_episodes=2000, save_path=None, 
                   config_name="default", verbose_freq=100):
    """Train REINFORCE agent."""
    
    episode_rewards = []
    episode_lengths = []
    wall_collision_rates = []
    scan_efficiencies = []
    pickup_attempts_list = []
    time_to_find_survivor_list = []
    loss_history = []
    recent_rewards = deque(maxlen=100)
    
    best_avg_reward = -np.inf
    
    for episode in range(1, num_episodes + 1):
        state, _ = env.reset()
        episode_reward = 0
        episode_length = 0
        done = False
        
        # Collect episode
        while not done:
            action = agent.select_action(state, training=True)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            
            agent.store_reward(reward)
            episode_reward += reward
            episode_length += 1
            state = next_state
        
        # Update policy
        loss = agent.update()
        
        # Track metrics from environment info
        episode_rewards.append(episode_reward)
        episode_lengths.append(info.get('episode_length', episode_length))
        wall_collision_rates.append(info.get('wall_collision_rate', 0))
        scan_efficiencies.append(info.get('scan_efficiency', 0))
        pickup_attempts_list.append(info.get('pickup_attempts', 0))
        
        time_found = info.get('time_to_find_survivor', None)
        if time_found is not None:
            time_to_find_survivor_list.append(time_found)
        
        recent_rewards.append(episode_reward)
        loss_history.append(loss)
        
        avg_reward = np.mean(recent_rewards)
        
        # Save best model
        if avg_reward > best_avg_reward and episode > 100:
            best_avg_reward = avg_reward
            if save_path:
                agent.save(save_path)
        
        # Logging
        if episode % verbose_freq == 0:
            recent_wall_collisions = np.mean(wall_collision_rates[-100:])
            recent_scan_eff = np.mean(scan_efficiencies[-100:])
            print(f"Episode {episode}/{num_episodes} | "
                  f"Avg Reward: {avg_reward:.2f} | "
                  f"Wall Collisions: {recent_wall_collisions:.2%} | "
                  f"Scan Eff: {recent_scan_eff:.2%} | "
                  f"Loss: {loss:.4f}")
    
    # Save final metrics
    metrics = {
        'config_name': config_name,
        'episode_rewards': episode_rewards,
        'episode_lengths': episode_lengths,
        'wall_collision_rates': wall_collision_rates,
        'scan_efficiencies': scan_efficiencies,
        'pickup_attempts': pickup_attempts_list,
        'time_to_find_survivor': time_to_find_survivor_list,
        'loss_history': loss_history,
        'best_avg_reward': best_avg_reward,
        'avg_wall_collision_rate': float(np.mean(wall_collision_rates)),
        'avg_scan_efficiency': float(np.mean(scan_efficiencies)),
        'avg_pickup_attempts': float(np.mean(pickup_attempts_list)),
        'avg_time_to_find_survivor': float(np.mean(time_to_find_survivor_list)) if time_to_find_survivor_list else None
    }
    
    return metrics

In [ ]:
# %% Cell 5: Hyperparameter Configurations
HYPERPARAMETER_CONFIGS = [
    {
        'name': 'config_1_baseline',
        'learning_rate': 0.001,
        'gamma': 0.99,
        'hidden_dims': [128, 64],
        'entropy_coef': 0.01,
        'num_episodes': 2000
    },
    {
        'name': 'config_2_high_lr',
        'learning_rate': 0.003,
        'gamma': 0.99,
        'hidden_dims': [128, 64],
        'entropy_coef': 0.01,
        'num_episodes': 2000
    },
    {
        'name': 'config_3_low_lr',
        'learning_rate': 0.0005,
        'gamma': 0.99,
        'hidden_dims': [128, 64],
        'entropy_coef': 0.01,
        'num_episodes': 2000
    },
    {
        'name': 'config_4_high_gamma',
        'learning_rate': 0.001,
        'gamma': 0.995,
        'hidden_dims': [128, 64],
        'entropy_coef': 0.01,
        'num_episodes': 2000
    },
    {
        'name': 'config_5_low_gamma',
        'learning_rate': 0.001,
        'gamma': 0.95,
        'hidden_dims': [128, 64],
        'entropy_coef': 0.01,
        'num_episodes': 2000
    },
    {
        'name': 'config_6_deep_network',
        'learning_rate': 0.001,
        'gamma': 0.99,
        'hidden_dims': [256, 128, 64],
        'entropy_coef': 0.01,
        'num_episodes': 2000
    },
    {
        'name': 'config_7_shallow_network',
        'learning_rate': 0.001,
        'gamma': 0.99,
        'hidden_dims': [64],
        'entropy_coef': 0.01,
        'num_episodes': 2000
    },
    {
        'name': 'config_8_high_entropy',
        'learning_rate': 0.001,
        'gamma': 0.99,
        'hidden_dims': [128, 64],
        'entropy_coef': 0.05,
        'num_episodes': 2000
    },
    {
        'name': 'config_9_no_entropy',
        'learning_rate': 0.001,
        'gamma': 0.99,
        'hidden_dims': [128, 64],
        'entropy_coef': 0.0,
        'num_episodes': 2000
    },
    {
        'name': 'config_10_optimal',
        'learning_rate': 0.002,
        'gamma': 0.99,
        'hidden_dims': [256, 128],
        'entropy_coef': 0.02,
        'num_episodes': 2000
    }
]

print(f"Total configurations to train: {len(HYPERPARAMETER_CONFIGS)}")

In [ ]:
# %% Cell 6: Train All Configurations
all_results = []

for i, config in enumerate(HYPERPARAMETER_CONFIGS, 1):
    print(f"\n{'='*70}")
    print(f"Training Configuration {i}/{len(HYPERPARAMETER_CONFIGS)}: {config['name']}")
    print(f"{'='*70}")
    print(f"Parameters: LR={config['learning_rate']}, γ={config['gamma']}, "
          f"Hidden={config['hidden_dims']}, Entropy={config['entropy_coef']}")
    
    # Create environment and agent
    env = FireRescueEnv(grid_size=10, max_time=200)
    agent = REINFORCEAgent(
        state_dim=env.obs_dim,
        action_dim=env.action_space.n,
        learning_rate=config['learning_rate'],
        gamma=config['gamma'],
        hidden_dims=config['hidden_dims'],
        entropy_coef=config['entropy_coef']
    )
    
    # Train
    save_path = f"../models/reinforce/{config['name']}.pth"
    metrics = train_reinforce(
        env, agent, 
        num_episodes=config['num_episodes'],
        save_path=save_path,
        config_name=config['name'],
        verbose_freq=200
    )
    
    # Add config to metrics
    metrics['config'] = config
    all_results.append(metrics)
    
    print(f"✓ Best Avg Reward: {metrics['best_avg_reward']:.2f}")
    print(f"✓ Avg Wall Collision Rate: {metrics['avg_wall_collision_rate']:.2%}")
    print(f"✓ Avg Scan Efficiency: {metrics['avg_scan_efficiency']:.2%}")
    print(f"✓ Avg Pickup Attempts: {metrics['avg_pickup_attempts']:.2f}")
    if metrics['avg_time_to_find_survivor']:
        print(f"✓ Avg Time to Find Survivor: {metrics['avg_time_to_find_survivor']:.1f} steps")
    
    env.close()

# Save all results (convert numpy types to native Python types)
def convert_to_json_serializable(obj):
    """Convert numpy types to native Python types for JSON serialization."""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_to_json_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_json_serializable(item) for item in obj]
    return obj

results_to_save = []
for r in all_results:
    result_copy = {k: v for k, v in r.items() if k != 'config'}
    results_to_save.append(convert_to_json_serializable(result_copy))

with open('../models/reinforce/all_results.json', 'w') as f:
    json.dump(results_to_save, f, indent=2)

print("\n" + "="*70)
print("ALL TRAINING COMPLETED!")
print("="*70)

In [ ]:
# %% Cell 7: Evaluation - Training Process Comparison
def plot_training_comparison(results):
    """Plot comparison of all training runs."""
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: Episode Rewards
    ax = axes[0, 0]
    for result in results:
        rewards = result['episode_rewards']
        smoothed = np.convolve(rewards, np.ones(50)/50, mode='valid')
        ax.plot(smoothed, label=result['config_name'], alpha=0.7)
    ax.set_xlabel('Episode')
    ax.set_ylabel('Smoothed Reward')
    ax.set_title('Episode Rewards Across Configurations')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Wall Collision Rate
    ax = axes[0, 1]
    for result in results:
        wall_collisions = result['wall_collision_rates']
        smoothed = np.convolve(wall_collisions, np.ones(50)/50, mode='valid')
        ax.plot(smoothed, label=result['config_name'], alpha=0.7)
    ax.set_xlabel('Episode')
    ax.set_ylabel('Wall Collision Rate')
    ax.set_title('Wall Collision Rate Across Configurations')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Plot 3: Scan Efficiency
    ax = axes[1, 0]
    for result in results:
        scan_eff = result['scan_efficiencies']
        smoothed = np.convolve(scan_eff, np.ones(50)/50, mode='valid')
        ax.plot(smoothed, label=result['config_name'], alpha=0.7)
    ax.set_xlabel('Episode')
    ax.set_ylabel('Scan Efficiency')
    ax.set_title('Scan Efficiency Across Configurations')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Plot 4: Policy Loss
    ax = axes[1, 1]
    for result in results:
        loss = result['loss_history']
        smoothed = np.convolve(loss, np.ones(50)/50, mode='valid')
        ax.plot(smoothed, label=result['config_name'], alpha=0.7)
    ax.set_xlabel('Episode')
    ax.set_ylabel('Policy Loss')
    ax.set_title('Policy Loss Across Configurations')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../models/reinforce/training_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_training_comparison(all_results)

In [ ]:
# %% Cell 8: Identify and Save Best Model
# Find best configuration
best_result = max(all_results, key=lambda x: x['best_avg_reward'])
best_config_name = best_result['config_name']

print(f"\n{'='*70}")
print(f"BEST CONFIGURATION: {best_config_name}")
print(f"{'='*70}")
print(f"Best Average Reward: {best_result['best_avg_reward']:.2f}")
print(f"Avg Wall Collision Rate: {best_result['avg_wall_collision_rate']:.2%}")
print(f"Avg Scan Efficiency: {best_result['avg_scan_efficiency']:.2%}")
print(f"Avg Pickup Attempts: {best_result['avg_pickup_attempts']:.2f}")
if best_result['avg_time_to_find_survivor']:
    print(f"Avg Time to Find Survivor: {best_result['avg_time_to_find_survivor']:.1f} steps")
print(f"\nHyperparameters:")
for key, value in best_result['config'].items():
    if key != 'name':
        print(f"  {key}: {value}")

# Copy best model to best_model folder
import shutil
best_model_src = f"../models/reinforce/{best_config_name}.pth"
best_model_dst = "../models/reinforce/best_model/best_reinforce.pth"
shutil.copy(best_model_src, best_model_dst)

# Save best config
with open('../models/reinforce/best_model/config.json', 'w') as f:
    json.dump(best_result['config'], f, indent=2)

print(f"\n✓ Best model saved to: {best_model_dst}")

In [ ]:
# %% Cell 9: Evaluation Cell 2 - Best Model Performance
def evaluate_best_model(model_path, config, num_episodes=100):
    """Evaluate the best model."""
    
    env = FireRescueEnv(grid_size=10, max_time=250)
    agent = REINFORCEAgent(
        state_dim=env.obs_dim,
        action_dim=env.action_space.n,
        learning_rate=config['learning_rate'],
        gamma=config['gamma'],
        hidden_dims=config['hidden_dims'],
        entropy_coef=config['entropy_coef']
    )
    agent.load(model_path)
    
    episode_rewards = []
    episode_lengths = []
    successes = []
    wall_collision_rates = []
    scan_efficiencies = []
    pickup_attempts_list = []
    time_to_find_survivor_list = []
    
    print("Calculating episode performance metrics...")
    for episode in range(num_episodes):
        state, _ = env.reset()
        episode_reward = 0
        done = False
        
        while not done:
            action = agent.select_action(state, training=False)
            state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            episode_reward += reward
        
        # Extract metrics from final info
        episode_rewards.append(episode_reward)
        episode_lengths.append(info.get('episode_length', 0))
        successes.append(1 if info.get('success', False) else 0)
        wall_collision_rates.append(info.get('wall_collision_rate', 0))
        scan_efficiencies.append(info.get('scan_efficiency', 0))
        pickup_attempts_list.append(info.get('pickup_attempts', 0))
        
        time_found = info.get('time_to_find_survivor', None)
        if time_found is not None:
            time_to_find_survivor_list.append(time_found)
        
        # Debug output for first 5 episodes
        if episode < 5:
            print(f"  Episode {episode+1}: "
                  f"Steps: {info.get('episode_length', 0)} | "
                  f"Wall Collisions: {info.get('wall_collision_rate', 0):.2%} | "
                  f"Scan Eff: {info.get('scan_efficiency', 0):.2%}")
    
    env.close()
    
    return {
        'mean_reward': np.mean(episode_rewards),
        'std_reward': np.std(episode_rewards),
        'mean_length': np.mean(episode_lengths),
        'std_length': np.std(episode_lengths),
        'success_rate': np.mean(successes),
        'wall_collision_rate': np.mean(wall_collision_rates),
        'scan_efficiency': np.mean(scan_efficiencies),
        'avg_pickup_attempts': np.mean(pickup_attempts_list),
        'avg_time_to_find_survivor': np.mean(time_to_find_survivor_list) if time_to_find_survivor_list else None,
        'survivor_found_count': len(time_to_find_survivor_list),
        'episode_rewards': episode_rewards,
        'episode_lengths': episode_lengths,
        'wall_collision_rates': wall_collision_rates,
        'scan_efficiencies': scan_efficiencies
    }

# Evaluate best model
eval_results = evaluate_best_model(
    best_model_dst, 
    best_result['config'],
    num_episodes=100
)

print(f"\n📊 Evaluation Summary:")
print(f"   Average Episode Length: {eval_results['mean_length']:.1f} steps")
print(f"   Wall Collision Rate: {eval_results['wall_collision_rate']:.2%}")
print(f"   Scan Efficiency: {eval_results['scan_efficiency']:.2%}")
print(f"   Avg Pickup Attempts: {eval_results['avg_pickup_attempts']:.2f}")
if eval_results['avg_time_to_find_survivor']:
    print(f"   Avg Time to Find Survivor: {eval_results['avg_time_to_find_survivor']:.1f} steps")
print(f"   Success Rate: {eval_results['success_rate']:.2%}")

print(f"\n{'='*70}")
print("BEST MODEL EVALUATION (100 Episodes)")
print(f"{'='*70}")
print(f"Mean Reward: {eval_results['mean_reward']:.2f} ± {eval_results['std_reward']:.2f}")
print(f"Mean Episode Length: {eval_results['mean_length']:.1f} ± {eval_results['std_length']:.1f}")
print(f"Success Rate: {eval_results['success_rate']:.2%}")
print(f"Wall Collision Rate: {eval_results['wall_collision_rate']:.2%}")
print(f"Scan Efficiency: {eval_results['scan_efficiency']:.2%}")

In [ ]:
# %% Cell 10: Best Model Visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Plot 1: Reward Distribution
ax = axes[0, 0]
ax.hist(eval_results['episode_rewards'], bins=30, alpha=0.7, color='blue', edgecolor='black')
ax.axvline(eval_results['mean_reward'], color='red', linestyle='--', 
           label=f"Mean: {eval_results['mean_reward']:.2f}")
ax.set_xlabel('Episode Reward')
ax.set_ylabel('Frequency')
ax.set_title('Best Model: Reward Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Episode Rewards Over Time
ax = axes[0, 1]
ax.plot(eval_results['episode_rewards'], alpha=0.6, label='Episode Reward')
ax.axhline(eval_results['mean_reward'], color='red', linestyle='--', 
           label=f"Mean: {eval_results['mean_reward']:.2f}")
ax.set_xlabel('Episode')
ax.set_ylabel('Reward')
ax.set_title('Best Model: Episode Rewards')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Episode Length Distribution
ax = axes[0, 2]
ax.hist(eval_results['episode_lengths'], bins=30, alpha=0.7, color='green', edgecolor='black')
ax.axvline(eval_results['mean_length'], color='red', linestyle='--',
           label=f"Mean: {eval_results['mean_length']:.1f}")
ax.set_xlabel('Episode Length')
ax.set_ylabel('Frequency')
ax.set_title('Best Model: Episode Length Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Wall Collision Rate Distribution
ax = axes[1, 0]
ax.hist(eval_results['wall_collision_rates'], bins=30, alpha=0.7, color='red', edgecolor='black')
ax.axvline(eval_results['wall_collision_rate'], color='darkred', linestyle='--',
           label=f"Mean: {eval_results['wall_collision_rate']:.2%}")
ax.set_xlabel('Wall Collision Rate')
ax.set_ylabel('Frequency')
ax.set_title('Best Model: Wall Collision Rate')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 5: Scan Efficiency Distribution
ax = axes[1, 1]
ax.hist(eval_results['scan_efficiencies'], bins=30, alpha=0.7, color='purple', edgecolor='black')
ax.axvline(eval_results['scan_efficiency'], color='darkviolet', linestyle='--',
           label=f"Mean: {eval_results['scan_efficiency']:.2%}")
ax.set_xlabel('Scan Efficiency')
ax.set_ylabel('Frequency')
ax.set_title('Best Model: Scan Efficiency')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 6: Success Rate Summary
ax = axes[1, 2]
metrics = ['Success\nRate', 'Wall\nCollisions', 'Scan\nEfficiency']
values = [eval_results['success_rate'], eval_results['wall_collision_rate'], eval_results['scan_efficiency']]
colors = ['green', 'red', 'purple']
bars = ax.bar(metrics, values, color=colors, alpha=0.7, edgecolor='black')
ax.set_ylabel('Rate')
ax.set_title('Best Model: Key Metrics Summary')
ax.set_ylim([0, 1])
# Add value labels on bars
for bar, value in zip(bars, values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{value:.1%}', ha='center', va='bottom', fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../models/reinforce/best_model/evaluation_plots.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ All plots saved successfully!")
print("✓ Training complete! Check '../models/reinforce/' for all saved models.")